# import

In [ ]:
import os
import csv
from scipy import stats
import radiomics
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from probatus.feature_elimination import ShapRFECV
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import shap
import lime
import lime.lime_tabular
from lime.lime_text import LimeTextExplainer
import seaborn as sns
from mrmr import mrmr_classif
import pandas as pd
import pingouin as pg
from pathlib import Path
import random

# load data

In [ ]:
icc_sysucc_muti_module = pd.read_csv('next_icc_sysucc_muti_module.csv')
icc_sysucc_id_label = icc_sysucc_muti_module.iloc[:,:3]
icc_sysucc_muti_module.head()

scaler = StandardScaler()
icc_sysucc_data_muti_module = icc_sysucc_muti_module.iloc[:,3:].copy()
icc_sysucc_data_muti_module = pd.DataFrame(scaler.fit_transform(icc_sysucc_data_muti_module), columns=icc_sysucc_data_muti_module.columns)
icc_sysucc_label_muti_module = icc_sysucc_id_label['label'].copy()

## set 10 groups

In [337]:
label_1_list = icc_sysucc_id_label[icc_sysucc_id_label['label']==1]['index_id'].tolist()
label_0_list = icc_sysucc_id_label[icc_sysucc_id_label['label']==0]['index_id'].tolist()

random.seed(seed)
random.shuffle(label_1_list)
random.shuffle(label_0_list)

def divide_t(s,S):
    N = list()
    for n in S:
        if not n in s:
            N.append(n)
    return(N)

test_id_1 = label_1_list[:4]+label_0_list[:7]
train_id_1 = divide_t(test_id_1,label_1_list+label_0_list)

test_id_2 = label_1_list[4:8]+label_0_list[7:14]
train_id_2 = divide_t(test_id_2,label_1_list+label_0_list)

test_id_3 = label_1_list[8:12]+label_0_list[14:21]
train_id_3 = divide_t(test_id_3,label_1_list+label_0_list)

test_id_4 = label_1_list[12:16]+label_0_list[21:28]
train_id_4 = divide_t(test_id_4,label_1_list+label_0_list)

test_id_5 = label_1_list[16:20]+label_0_list[28:35]
train_id_5 = divide_t(test_id_5,label_1_list+label_0_list)

test_id_6 = label_1_list[20:24]+label_0_list[35:42]
train_id_6 = divide_t(test_id_1,label_1_list+label_0_list)

test_id_7 = label_1_list[24:28]+label_0_list[42:49]
train_id_7 = divide_t(test_id_2,label_1_list+label_0_list)

test_id_8 = label_1_list[28:31]+label_0_list[49:57]
train_id_8 = divide_t(test_id_3,label_1_list+label_0_list)

test_id_9 = label_1_list[31:34]+label_0_list[57:65]
train_id_9 = divide_t(test_id_4,label_1_list+label_0_list)

test_id_10 = label_1_list[34:]+label_0_list[65:]
train_id_10 = divide_t(test_id_5,label_1_list+label_0_list)    

gsk = list()
group_x_y = list()
group_x_y.append(np.array(train_id_1))
group_x_y.append(np.array(test_id_1))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_2))
group_x_y.append(np.array(test_id_2))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_3))
group_x_y.append(np.array(test_id_3))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_4))
group_x_y.append(np.array(test_id_4))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_5))
group_x_y.append(np.array(test_id_5))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_6))
group_x_y.append(np.array(test_id_6))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_7))
group_x_y.append(np.array(test_id_7))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_8))
group_x_y.append(np.array(test_id_8))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_9))
group_x_y.append(np.array(test_id_9))
gsk.append(group_x_y)

group_x_y = list()
group_x_y.append(np.array(train_id_10))
group_x_y.append(np.array(test_id_10))
gsk.append(group_x_y)

## fit() CV

In [ ]:
clf = XGBClassifier(booster ='gbtree',
                    reg_alph = 0.5,
                    reg_lambda=1,
                    gamma=0,
                    eval_metric = 'auc', 
                    objective = 'binary:logistic')

param_grid = {
    "n_estimators": [x for x in range(30,60,2)],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8],
    'learning_rate':[0.1, 0.08, 0.05],
    'max_depth':[3, 4, 5, 7, 8, 9, 10, 11, 12],
    'random_state':[x for x in range(10,20)],
    'min_child_weigh':[1, 2, 3, 4],
    'subsample':[0.2],
}

search = RandomizedSearchCV(clf, param_grid, random_state=seed_search)

shap_elimination = ShapRFECV(clf=search, step=0.2, cv=gsk, scoring="roc_auc")
report = shap_elimination.fit_compute(icc_sysucc_data_muti_module, icc_sysucc_label_muti_module)
result_df = shap_elimination.report_df
index_max_id = result_df['val_metric_mean'].idxmax()
shap_features_number = result_df.loc[index_max_id]['num_features']
shap_features_need = shap_elimination.get_reduced_features_set(num_features= int(shap_features_number))

In [ ]:
clf = XGBClassifier(booster ='gbtree',
                    reg_alph = 0.5,
                    reg_lambda=1,
                    gamma=0,
                    eval_metric = 'auc', 
                    objective = 'binary:logistic')

estimator = RandomizedSearchCV(clf, param_grid, cv=gsk, scoring="roc_auc",random_state=seed_search)
estimator.fit(icc_sysucc_data_muti_module[shap_features_need], icc_sysucc_label_muti_module)
params_need = estimator.best_params_

In [ ]:
model_final = XGBClassifier(max_depth = params_need['max_depth'],
                            n_estimators = params_need['n_estimators'], 
                            learning_rate = params_need['learning_rate'],
                            colsample_bytree = params_need['colsample_bytree'],
                            min_child_weigh = params_need['min_child_weigh'],
                            random_state = params_need['random_state'],
                            subsample= params_need['subsample'],
                            booster ='gbtree',
                            reg_alph = 0.5,
                            reg_lambda=1,
                            gamma=0,
                            eval_metric = 'auc', 
                            objective = 'binary:logistic')

model_final.fit(icc_sysucc_data_muti_module[shap_features_need], icc_sysucc_label_muti_module, 
          eval_set=[(icc_sysucc_data_muti_module[shap_features_need], icc_sysucc_label_muti_module)])

model_final.save_model('final_model.json')

# external test for final model

In [ ]:
icc_fuyi_muti_module = pd.read_csv('next_icc_fuyi_muti_module.csv')
icc_nanfang_muti_module = pd.read_csv('next_icc_nanfang_muti_module.csv')

icc_external_data_muti_module = pd.concat([icc_fuyi_muti_module,icc_nanfang_muti_module],axis=0).iloc[:,3:]
icc_external_label_muti_module= pd.concat([icc_fuyi_muti_module,icc_nanfang_muti_module],axis=0)['label']
icc_external_id_label_muti_module= pd.concat([icc_fuyi_muti_module,icc_nanfang_muti_module],axis=0).iloc[:,:3]

icc_external_data_muti_module = pd.DataFrame(scaler.fit_transform(icc_external_data_muti_module), columns=icc_external_data_muti_module.columns)

icc_external_data_muti_module.head()

In [ ]:
y_test = icc_external_label_muti_module.tolist()
pred_pro = model_final.predict_proba(icc_external_data_muti_module[shap_features_need])[:,1]

csv_detail_ex_name = 'csv_detail_ex.csv'
csv_detail_ex = icc_external_id_label_muti_module.copy()
csv_detail_ex['predict_proba'] = pred_pro.tolist()
csv_detail_ex.to_csv(csv_detail_ex_name, header=True, index=False)

# shap explainer

In [ ]:
explainer = shap.TreeExplainer(model_final,icc_sysucc_data_muti_module[shap_features_need],link= shap.links.logit, model_output='probability')
shap_values = explainer(icc_external_data_muti_module[shap_features_need],icc_external_label_muti_module)

In [ ]:
plt.figure()
shap.plots.beeswarm(shap_values,max_display=25, show=False)
plt.savefig('shap_ex.svg', dpi=300, bbox_inches = 'tight')
plt.close()

In [ ]:
i = 30 # select a patient id
plt.figure()
shap.plots.waterfall(shap_values[i],max_display=11,show=False)
plt.savefig('shap_ex['+str(i)+']_waterfall.svg', dpi=300, bbox_inches = 'tight')
plt.close()

In [ ]:
pd_shap_values = pd.DataFrame(shap_values.values,columns = shap_features_need, index = icc_external_id_label_muti_module.index)
pd_shap_values.to_csv('pd_shap_values_ex.csv',header=True, index=True)